4He Groundstate calculation with fixed a, beta, gamma

In [4]:
# Packages
import numpy as np
import matplotlib.pyplot as plt
from numba import njit

In [5]:
#parameters
hbar2div2m = 20.74  # MeV * fm^2

In [6]:
# Define the inter-nucleon potential function
@njit
def potential(r2):
    return (1000.0 * np.exp(-3.0 * r2) 
            - 163.35 * np.exp(-1.05 * r2) 
            - 83.0 * np.exp(-0.8 * r2) 
            - 21.5 * np.exp(-0.6 * r2) 
            - 11.5 * np.exp(-0.4 * r2))

In [7]:
# Trial wave function for 4He
@njit
def trial_wave_function(positions, a, beta, gamma):
    n_particles = positions.shape[0]
    psi = 1.0
    for i in range(n_particles):
        for j in range(i + 1, n_particles):
            r2 = np.sum((positions[i] - positions[j]) ** 2)
            correlation = np.exp(-gamma * r2) - a * np.exp(-(beta + gamma) * r2)
            psi *= correlation
    return psi

In [8]:
# Local energy calculation
@njit
def local_energy(positions, a, beta, gamma):
    n_particles = positions.shape[0]
    # Potential part
    V = 0.0
    for i in range(n_particles):
        for j in range(i + 1, n_particles):
            r2 = np.sum((positions[i] - positions[j]) ** 2)
            V += potential(r2)

    # Kinetic energy part
    psi = trial_wave_function(positions, a, beta, gamma)
    grad_psi = 0.0 # Initializing wave function gradient
    delta = 1e-5  # Delta size for kinetic energy approximate calculation

    # Evaluating psi in two points and take the difference
    for i in range(n_particles):
        for j in range(3):  # They are 3D coordinates
            positions[i, j] += delta
            psi_plus = trial_wave_function(positions, a, beta, gamma)
            positions[i, j] -= 2 * delta
            psi_minus = trial_wave_function(positions, a, beta, gamma)
            positions[i, j] += delta  # Restore original position
            
            grad_psi += (psi_plus - 2 * psi + psi_minus) / (delta ** 2)
    
    kinetic_energy = -hbar2div2m * grad_psi / psi
    return kinetic_energy + V

In [9]:
# Main Variational Monte Carlo function with parameter optimization
@njit
def variational_monte_carlo(n_particles=4, n_steps=1000, thermalization_steps=300, a=0.1, beta=0.1, gamma=0.1):
    positions = np.random.normal(0, 5, (n_particles, 3))  # Random initial positions
    energy_samples = []

    # Thermalization phase (accepted steps without recording the resulting energy)
    for _ in range(thermalization_steps):
        # Proposing new positions
        new_positions = positions + np.random.normal(0, 0.5, positions.shape)
        psi_old = trial_wave_function(positions, a, beta, gamma)
        psi_new = trial_wave_function(new_positions, a, beta, gamma)
        
        # Define the metropolis acceptance condition
        acceptance_ratio = (psi_new / psi_old) ** 2
        if np.random.rand() < acceptance_ratio:
            positions = new_positions  # Accepting step
    
    # Main sampling phase
    for step in range(n_steps):
        # Proposing new positions
        new_positions = positions + np.random.normal(0, 0.5, positions.shape)
        psi_old = trial_wave_function(positions, a, beta, gamma)
        psi_new = trial_wave_function(new_positions, a, beta, gamma)
        
        # Define the metropolis acceptance condition
        acceptance_ratio = (psi_new / psi_old) ** 2
        if np.random.rand() < acceptance_ratio:
            positions = new_positions  # Accepting step
        
        E_local = local_energy(positions, a, beta, gamma)
        energy_samples.append(E_local)

    # Converting energy_samples to a NumPy array, for compatibility issues with numba.
    energy_samples = np.array(energy_samples)

    # Computing the mean energy - and it's variance - over all the VMC steps
    mean_energy = np.mean(energy_samples)
    std_error = np.std(energy_samples) / np.sqrt(n_steps)
    
    return mean_energy, std_error


In [10]:
# Fixed parameters from 
a_fixed = 0.7191
gamma_fixed = 0.08597
beta_fixed = 2.13796 - gamma_fixed

# Computing ground state energy with VMC
mean_energy, std_error = variational_monte_carlo(n_particles=4, n_steps=2000, thermalization_steps=300, a=a_fixed, beta=beta_fixed, gamma=gamma_fixed)


In [17]:
# Print the energy and the standard deviation
print("Estimated ground state energy:", mean_energy, "MeV")
print("with standard error:", std_error, "MeV")

Estimated ground state energy: -26.055235471122664 MeV
with standard error: 0.6317051515816682 MeV
